# ***Data Extraction and Processing from ERA5***

### **Overview**

This project began by obtaining climate reanalysis data from the **ERA5 dataset**, provided by the **European Centre for Medium-Range Weather Forecasts** **(ECMWF)**. The dataset contains hourly estimates of various atmospheric, land, and oceanic climate variables dating back to 1940.

### **Accessing the ERA5 Dataset**

To download the necessary data, we followed these steps:

Create an account on the ECMWF

1.  Create an account on the ECMWF
website: https://www.ecmwf.int/en/forecasts/dataset/ecmwf-reanalysis-v5

2.   Navigate to the database selection area
website:https://cds.climate.copernicus.eu/datasets

3. Select the specific database:
website:https://cds.climate.copernicus.eu/datasets/reanalysis-era5-single-levels?tab=overview

4. Choose the required parameters, submit a download request, and wait for approval.

### **Data Format and Preprocessing**

The downloaded data was provided in GRIB format, which required specialized tools and libraries for decoding and processing. To facilitate further analysis, we:


*  Uploaded the dataset to a cloud drive, enabling seamless access in Google Colab.


*   Implemented the required Python libraries for reading and processing the GRIB files.



*   Converted the extracted data into a CSV file for easier manipulation and storage.








### **Parameter Selection and Extraction**

We developed two separate extraction scripts based on the structure of the ERA5 dataset:

**1. Radiation Parameters Extraction**
In ERA5, radiation-related parameters are aggregated, requiring specialized processing. The extracted parameters include:
*   SSRD: Surface Solar Radiation Downwards
*   STR: Surface Net Thermal Radiation
*  OLR: Mean Surface Net Long-Wave Radiation Flux

**2. General Parameter Extraction**
A second script was designed to extract other meteorological parameters, including:



*   FA: Forecast Albedo
*   TCC: Total Cloud Cover
* 2mtemperature: 2m Temperature
*   2mdewpointtemperature: 2m Dewpoint Temperature
*   Surfacepressure: Surface Pressure

These parameters are essential for our energy absorption calculations and further climate analysis.

In [ ]:
# Install necessary libraries
!apt-get install -y libeccodes-dev  # Installs ECCODES library required for GRIB file handling
!pip install pygrib  # Library for reading GRIB files
!pip install pandas  # Data manipulation library
!pip install pygrib pandas  # Ensures pygrib and pandas are installed


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libeccodes-data libeccodes0
The following NEW packages will be installed:
  libeccodes-data libeccodes-dev libeccodes0
0 upgraded, 3 newly installed, 0 to remove and 18 not upgraded.
Need to get 3,076 kB of archives.
After this operation, 60.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libeccodes-data all 2.24.2-1 [1,592 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libeccodes0 amd64 2.24.2-1 [614 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libeccodes-dev amd64 2.24.2-1 [870 kB]
Fetched 3,076 kB in 3s (1,153 kB/s)
Selecting previously unselected package libeccodes-data.
(Reading database ... 124950 files and directories currently installed.)
Preparing to unpack .../libeccodes-data_2.24.2-1_all.deb ...
Unpacking libeccodes-data (2.24.2-1) ...

In [ ]:
# Import required libraries
import pygrib  # Library for reading GRIB files
import pandas as pd  # Library for handling data structures like DataFrame
from google.colab import drive, files  # For mounting Google Drive and downloading files
import numpy as np  # Numerical computation library
from datetime import datetime  # For handling date and time operations


# Mount Google Drive to access stored files
drive.mount('/content/drive')


# Define the path to the GRIB file (change this path as needed)
grib_file_path = '/content/drive/My Drive/data_project/FA_2021.grib'


# Open the GRIB file
grbs = pygrib.open(grib_file_path)


# Initialize an empty list to store the data
data = []


# Define the grid resolution (in degrees) for latitude and longitude
# This helps in mapping data onto a structured grid
grid_resolution = 5.0  # 5-degree increments
lats_grid = np.arange(-90, 90, grid_resolution)  # Latitude grid points
lons_grid = np.arange(-180, 180, grid_resolution)  # Longitude grid points


# Set batch processing parameters to manage memory usage
batch_size = 10  # Number of GRIB messages processed in one batch
batch_data = []  # Temporary storage for batch processing


# Define a reference start datetime for calculating time differences
start_datetime = datetime(2020, 1, 1, 0, 0, 0)


# Loop through each message in the GRIB file
for idx, grb in enumerate(grbs):
    name = grb.name  # Get the name of the parameter
    valid_date = grb.validDate  # Extract valid date of the GRIB message
    data_date = grb.dataDate  # Extract data date
    data_time = grb.dataTime  # Extract time information

    # Extract the data matrix (values) along with the corresponding latitude and longitude grids
    values, lats, lons = grb.data(lat1=-90, lat2=90, lon1=-180, lon2=180)
    lats = np.array(lats)  # Convert latitude values to NumPy array
    lons = np.array(lons)  # Convert longitude values to NumPy array
    values = np.array(values)  # Convert data values to NumPy array

    num_lats, num_lons = values.shape  # Get the shape of the extracted data

    # Loop through the predefined latitude and longitude grid
    for lat_grid_idx, lat in enumerate(lats_grid):
        for lon_grid_idx, lon in enumerate(lons_grid):
            # Find the closest data point in the GRIB file to match the grid point
            lat_idx = (np.abs(lats[:, 0] - lat)).argmin()  # Get closest latitude index
            lon_idx = (np.abs(lons[0, :] - lon)).argmin()  # Get closest longitude index

            # Ensure indices are within bounds before extracting the value
            if 0 <= lat_idx < num_lats and 0 <= lon_idx < num_lons:
                value = values[lat_idx, lon_idx]  # Extract the corresponding value

            else:
                value = np.nan  # Assign NaN if index is out of bounds
                print(f"Warning: Indices out of bounds for lat={lat}, lon={lon}. Setting value to NaN.")

            # Calculate time difference in minutes from the start datetime
            valid_datetime = valid_date.replace(hour=data_time // 100, minute=data_time % 100)
            time_difference = (valid_datetime - start_datetime).total_seconds() / 60

            # Extract the hour component from the time data
            current_hour = data_time // 100

            # Store extracted data in the batch list
            batch_data.append({
                'time_difference': time_difference,  # Time difference in minutes
                'value': round(value, 3),  # Extracted radiation value (rounded)
                'latitudepoint': round(lat, 2),  # Latitude coordinate
                'longitudepoint': round(lon, 2)  # Longitude coordinate
            })

    # Append processed batch data to the main data list and clear batch storage
    if (idx + 1) % batch_size == 0:
        data.extend(batch_data)
        batch_data.clear()

# Append any remaining batch data
if batch_data:
    data.extend(batch_data)

# Create a DataFrame from the collected data
df = pd.DataFrame(data)

# Ensure the DataFrame is not empty before sorting
if not df.empty:
    # Sort by time_difference to ensure the data is sorted by time
    df = df.sort_values(by=['time_difference'])
else:
    print("DataFrame is empty. Check your data extraction logic.")


# Extract the parameter name for file naming purposes
parameter_name = name if 'name' in locals() else 'Unknown Parameter'

# Define output file name and save DataFrame to CSV
output_file = f'/content/{parameter_name}_FA_2021.csv'
with open(output_file, 'w') as f:
   ## f.write(f'{parameter_name}\n') # Write parameter name in the first line
    df.to_csv(f, index=False, columns=['time_difference', 'value', 'latitudepoint', 'longitudepoint'])


# Download the output file to the local system
files.download(output_file)

# Print confirmation message
print(f"Data successfully saved to CSV file: {output_file}")


Mounted at /content/drive


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Data successfully saved to CSV file: /content/Forecast albedo_FA_2021.csv
